In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [4]:
pd.set_option('max_rows', 300)
read_pkl_gzip('../output/feature_importances/20190908_2116__CV0.9308086442211762__feature793.gz').sort_values(by='imp_avg', ascending=False).head(300)

,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,imp_fold6,imp_avg
feature,,,,,,,
507__C1-C14__ratio,111506.015625,116273.203125,115317.875000,118606.585938,114792.468750,110476.367188,114495.414062
507__C11-C14__ratio__ProductCD-C,4546.506348,10128.210938,9967.177734,9700.381836,10159.054688,10092.742188,9099.012695
509__C13-V308__ratio__ProductCD-W,10582.199219,7484.069336,7218.455078,7885.680176,7784.571777,7302.713379,8042.948730
509__C14-V317__ratio__ProductCD-W,11274.080078,7214.745605,6901.948730,7603.354004,7124.579590,7500.880371,7936.597656
509__C13-V45__ratio__ProductCD-C,7632.777344,6810.592285,6868.668457,4086.564697,5808.923828,7773.004395,6496.755371
508__C2-D1__ratio,4443.493652,4779.999023,4681.280762,4577.054199,4641.958008,5118.517578,4707.050293
508__C13-D1__ratio,2752.022461,5005.639648,4869.914062,4816.844727,5125.154297,4144.936035,4452.418457
601__ProductCD-all_day-62_date_count,938.976807,5853.044434,5784.395508,5602.548340,6164.108398,1132.104004,4245.862793
507__C1-C14__diff,4051.194336,4451.426758,2383.454102,4089.188965,4376.420410,5127.178711,4079.810547


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Amt')
               or path.count('D')
               or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Amt')
               or path.count('D')
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
if COLUMN_ID in data.columns:
    data.set_index(COLUMN_ID, inplace=True)

base_train = read_pkl_gzip('../input/base_train.gz').set_index(COLUMN_ID)
base_test = read_pkl_gzip('../input/base_test.gz').set_index(COLUMN_ID)
base = pd.concat([base_train, base_test], axis=0)

cols_num = get_numeric_features(data, COLUMNS_IGNORE)
cols_num = [col for col in cols_num if col.count('C') or col.count('D')]
data[cols_num] = data[cols_num].astype('float32')

df_user_id_ca = pd.read_csv('../output/same_user_pattern/0903__same_user_id__card_addr.csv').set_index(COLUMN_ID)
df_user_id_cap = pd.read_csv('../output/same_user_pattern/0903__same_user_id__card_addr_pemail.csv').set_index(COLUMN_ID)
df_user_id_capm = pd.read_csv('../output/same_user_pattern/0902__same_user_id__card_addr_pemail_M.csv').set_index(COLUMN_ID)
df_user_id_bear = pd.read_csv('../output/same_user_pattern/20190901_user_ids_share.csv').set_index(COLUMN_ID)

data['user_id_card_addr'] = df_user_id_ca['predicted_user_id']
data['user_id_card_addr_pemail'] = df_user_id_cap['predicted_user_id']
data['user_id_card_addr_pemail_M'] = df_user_id_capm['predicted_user_id']
data['user_id_bear'] = df_user_id_bear['predicted_user_id']

In [3]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())

In [4]:
#========================================================================
# ProductCDあたりのC
#========================================================================
cols_Amt = [col for col in data.columns if col.count('Amt')]
cols_pcd = data['ProductCD'].unique()

for col in tqdm(cols_Amt):
    for pcd in cols_pcd:
        feature_name = f'{col}__ProductCD-{pcd}'
        data[feature_name] = -1
        data.loc[data['ProductCD'].isin([pcd]), feature_name] = data.loc[data['ProductCD'].isin([pcd]), col]

100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


In [ ]:
#========================================================================
# FE Aggregation User ID & TimeSeries Date
#========================================================================

def parallel_agg(df, base_key, base_date, n_day, feature):
    result = df.groupby(base_key)[feature].agg({
        f'{base_key}_day{n_day}_{feature}_sum': 'sum',
        f'{base_key}_day{n_day}_{feature}_min': 'max',
        f'{base_key}_day{n_day}_{feature}_mean': 'mean',
    })
    return result

# User別に期間を切って集計できる様にする
cols_feature = [col for col in data.columns if col.count('Amt') and col.count('Product')]
col_amt = 'TransactionAmt'
dir_save = 'valid'
df = data
list_base_key = [col for col in df.columns if col.count('user_id')]
list_end_date = sorted(df['date'].unique())[1:]
list_base_date = list_end_date
list_n_day = [1, 3, 5, 7, 10, 14, 21, 28, 31, 62, 93, 124, 180][1:]
# list_n_day = [1, 3, 5, 7, 10, 14]

for base_key in list_base_key:
    base_train = data[[base_key, 'date', col_amt]].iloc[:len(df_train)]
    base_test  = data[[base_key, 'date', col_amt]].iloc[len(df_train):]
    for n_day in list_n_day:
        list_term_df = []
        for end_date in tqdm(list_end_date):
            start_date = date_add_days(end_date, n_day*-1)
            tmp = data[(start_date <= data.date) & (data.date <= end_date)]
            tmp_user = data[data['date']==end_date][[base_key]].drop_duplicates()
            list_p = Parallel(len(cols_feature))([delayed(parallel_agg)(tmp[[base_key, feature]], base_key, end_date, n_day, feature) for feature in cols_feature])
            tmp_df_date = pd.concat(list_p, axis=1).reset_index()
            tmp_user = tmp_user.merge(tmp_df_date, on=base_key, how='inner')
            tmp_user['date'] = end_date
            list_term_df.append(tmp_user)
            
        df_agg = pd.concat(list_term_df, axis=0)
        base_train_agg = base_train.merge(df_agg, how='left', on=[base_key, 'date'])
        base_test_agg = base_test.merge(df_agg, how='left', on=[base_key, 'date'])
        
        cols_agg = [col for col in base_train_agg.columns if col.count(f'{base_key}_day')]
        
        for col in cols_agg:
            base_train_agg[f"{col}_org_ratio"] =  base_train_agg[col] / (base_train_agg[col_amt]+10)
            base_train_agg[f"{col}_org_diff"] =  base_train_agg[col] - (base_train_agg[col_amt])
            base_test_agg[f"{col}_org_ratio"] =  base_test_agg[col] / (base_test_agg[col_amt]+10)
            base_test_agg[f"{col}_org_diff"] =  base_test_agg[col] - (base_test_agg[col_amt])
            
        cols_save = [col for col in base_train_agg.columns if col.count(f'org_ratio') or col.count('org_diff')]
        
        save_feature(base_train_agg[cols_save], '505', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test_agg[cols_save],  '505', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

In [51]:
#========================================================================
# FE Aggregation User ID & TimeSeries Hour
#========================================================================


In [13]:
# for df, agg_cols in zip(arg_df_list, arg_list):
def parallel_agg(df, agg_cols):
    error_keys = []
    error_cols = []
    for col in agg_cols:
        aggs = {}
#         aggs[col] = ['mean', 'max', 'min', 'std']
        aggs[col] = ['max', 'min']
        
        for key in list_key:
            
            tmp_base_train = base_train.join(df[key])
            tmp_base_test = base_test.join(df[key])
            
            try:
                base_agg = df[key].to_frame()
            except AttributeError:
                error_keys.append(key)
                error_cols += agg_cols
            
            df_agg = df.groupby(key).agg(aggs)
            df_agg.columns = get_new_columns(key+'_', aggs)
            max_col = [col for col in df_agg.columns if col.count('_max')][0]
            min_col = [col for col in df_agg.columns if col.count('_min')][0]
            df_agg[max_col+'_min_diff'] = df_agg[max_col] - df_agg[min_col]
            
            df_agg.drop([max_col, min_col], axis=1, inplace=True)
            
            df_agg.reset_index(inplace=True)
            
            base_train_agg = tmp_base_train.merge(df_agg, on=key, how='left')
            base_test_agg = tmp_base_test.merge(df_agg, on=key, how='left')
            
            del df_agg, tmp_base_train, tmp_base_test
            gc.collect()
            
            print(base_train_agg.shape, base_test_agg.shape)
            cols_feature = [
                col for col in base_train_agg.columns 
                if col not in COLUMNS_IGNORE and col != key and col != 'D1']
            save_feature(base_train_agg[cols_feature], '502', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
            save_feature(base_test_agg[cols_feature],  '502', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)
            
            del base_train_agg, base_test_agg
            gc.collect()

In [14]:
err = Parallel(n_jobs)([delayed(parallel_agg)(d, ac) for d, ac in zip(arg_df_list, arg_list)])

In [ ]:
df_feat.head()